In [5]:
# Setup
import pandas as pd
import sklearn
import re
import numpy as np

diabetes_df = pd.read_csv("./data/diabetes_df.csv")

/Volumes/EXTERNAL/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
readmitted_outcome = [0 if x == 'NO' else 1 for x in diabetes_df['readmitted']]
first_number_regex = re.compile(r'^\[(\d*)') # Matches a bracket followed by numbers (0 or more, greedy)

def extract_minimum_age(age):
    matching_number = first_number_regex.match(age).group(1) # first matching group only extract the number
    return int(matching_number)

In [7]:
diabetes_df['age'].apply(extract_minimum_age)
model_subset = diabetes_df.loc[:, ['encounter_id', 'race', 'gender', 'num_lab_procedures', 'num_procedures', 'num_medications', 'number_outpatient', 'number_emergency', 'number_inpatient', 'number_diagnoses', 'time_in_hospital']].copy()
id_subset = diabetes_df.loc[:, ['encounter_id', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id']].copy()
ccs_subset = diabetes_df.loc[:, ['encounter_id', 'CCS Category Description 1', 'CCS Category Description 2', 'CCS Category Description 3']].copy()
model_subset = pd.get_dummies(model_subset, prefix = "ind_", dummy_na = True, drop_first = True)
id_subset = pd.get_dummies(id_subset, columns = ['admission_type_id', 'discharge_disposition_id', 'admission_source_id'],prefix = {x:x for x in id_subset.columns if x != 'encounter_id'}, dummy_na = True, drop_first = True)
ccs_subset = pd.get_dummies(ccs_subset, prefix = "ind_", dummy_na = True, drop_first = True)
ccs_subset = ccs_subset.groupby(ccs_subset.columns, axis = 1).sum()
model_dataset = (model_subset.merge(id_subset, how = "left", on = "encounter_id")
                             .merge(ccs_subset, how = "left", on = "encounter_id"))

In [8]:
model_dataset

,encounter_id,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,time_in_hospital,ind__Asian,...,ind__Ulcerat col,ind__Unclassified,ind__Urin stone,ind__Uterus cancr,ind__Varicose vn,ind__Viral infect,ind__Wht blood dx,ind__chf;nonhp,ind__gu cong anom,ind__nan_y
0,2278392,41,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2
1,149190,59,0,18,0,0,0,9,3,0,...,0,0,0,0,0,0,0,0,0,0
2,64410,11,5,13,2,0,1,6,2,0,...,0,0,0,0,0,0,0,0,0,2
3,500364,44,1,16,0,0,0,7,2,0,...,0,0,0,0,0,0,0,0,0,0
4,16680,51,0,8,0,0,0,5,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,51,0,16,0,0,0,9,3,0,...,0,0,0,0,0,0,0,0,0,0
101762,443847782,33,3,18,0,0,1,9,5,0,...,0,0,0,0,0,0,0,0,0,0
101763,443854148,53,0,9,1,0,0,13,1,0,...,0,0,0,0,0,0,0,0,0,2
101764,443857166,45,2,21,0,0,1,9,10,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
model_dataset['readmission'] = [0 if x == 'NO' else 1 for x in diabetes_df['readmitted']]


In [12]:
model_dataset

,encounter_id,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,time_in_hospital,ind__Asian,...,ind__Unclassified,ind__Urin stone,ind__Uterus cancr,ind__Varicose vn,ind__Viral infect,ind__Wht blood dx,ind__chf;nonhp,ind__gu cong anom,ind__nan_y,readmission
0,2278392,41,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,2,0
1,149190,59,0,18,0,0,0,9,3,0,...,0,0,0,0,0,0,0,0,0,1
2,64410,11,5,13,2,0,1,6,2,0,...,0,0,0,0,0,0,0,0,2,0
3,500364,44,1,16,0,0,0,7,2,0,...,0,0,0,0,0,0,0,0,0,0
4,16680,51,0,8,0,0,0,5,1,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101761,443847548,51,0,16,0,0,0,9,3,0,...,0,0,0,0,0,0,0,0,0,1
101762,443847782,33,3,18,0,0,1,9,5,0,...,0,0,0,0,0,0,0,0,0,0
101763,443854148,53,0,9,1,0,0,13,1,0,...,0,0,0,0,0,0,0,0,2,0
101764,443857166,45,2,21,0,0,1,9,10,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = (train_test_split(model_dataset
                                                     .drop(['encounter_id', 'readmission'], axis=1),
                                                     model_dataset['readmission'], 
                                                     test_size=0.2))

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

RandomForestClassifier()

In [19]:
prediction_probabilities = rf.predict_proba(X_test)[:, 1]

In [20]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

In [21]:
roc_auc_score(y_test, prediction_probabilities)

0.6844735864223745

In [22]:
average_precision_score(y_test, prediction_probabilities)

0.6386449612018049